In [1]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import wandb
import yaml
from torch.optim.lr_scheduler import StepLR
from utils.logger import *
from utils.models_init import *
from utils.training import * 
from utils.optimizers import *
from utils.custom_models import *

In [ ]:
with open('/home/disi/ml/intromlproject/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

config = config['config']
config['data_dir'] = config['data_dir'].format(root=config['root'], img_folder=config['img_folder'])
config['save_dir'] = config['save_dir'].format(root=config['root'], model_name=config['model_name'], img_folder=config['img_folder'])
if config['checkpoint'] is not None:
    config['checkpoint'] = config['checkpoint'].format(root=config['root'])
config['device'] = "cuda" if torch.cuda.is_available() else "cpu"
config['project_name'] = config['project_name'].format(model_name=config['model_name'])
config['dataset_name'] = config['dataset_name'].format(img_folder=config['img_folder'])

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((config['image_size'], config['image_size'])),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  
    transforms.ToTensor(),
    transforms.Normalize(mean=config['mean'], std=config['std'])
])

val_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [ ]:
train_dataset = datasets.ImageFolder(root=os.path.join(config['data_dir'], 'train'), transform=train_transform)
val_dataset = datasets.ImageFolder(root=os.path.join(config['data_dir'], 'val'), transform=train_transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=8)

In [ ]:
num_classes = len(train_loader.dataset.classes)
model = init_model(config['model_name'], num_classes)

/home/disi/ml/mlvenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/disi/ml/mlvenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
# Optimize only the parameters that require gradients
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)

In [ ]:
from torchvision import datasets, transforms
import os

# Define your data directory
data_dir = '/home/disi/ml/datasets/aircraft'

# Optionally, define transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Load datasets
datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'val']
}

In [ ]:
# Get the size of each dataset
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val']}
dataset_sizes

{'train': 3334, 'val': 3333}

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for inputs, labels in train_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

for inputs, labels in val_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)


In [ ]:
wandb.login()  # @edit
wandb.init(project=config['project_name'],
name = config['dataset_name'],
#name=f"{config['model_name']}_{config['dataset_name']}_opt: {config['optimizer']}_batch_size: {config['batch_size']}_lr: {config['learning_rate']}",
config=config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: lorenzo-chicco (lorenzochicco99). Use `wandb login --relogin` to force relogin


In [ ]:
checkpoint_path = config['save_dir']
checkpoint = config['checkpoint']

In [ ]:
if checkpoint is not None:
    load_checkpoint(model, checkpoint_path, device="cuda")
    print("Checkpoint loaded from" + checkpoint_path)

In [ ]:
model_name = config['model_name']
dataset_name = config['model_name']
save_dir = config['save_dir']

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25):
    counter = 0
    patience = 3
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)


        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                data_loader = train_loader
                print("Entered train mode")

            else:
                model.eval()   # Set model to evaluate mode
                data_loader = val_loader
                print("Entered val mode")

            running_loss = 0.0
            running_corrects = 0
            total_samples = 0

            # Iterate over data
            for inputs, labels in data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                total_samples += inputs.size(0)

            epoch_loss = running_loss / total_samples
            epoch_acc = running_corrects.double() / total_samples

            if phase == "train":
                wandb.log({
                "train/loss": epoch_loss,
                "train/accuracy":epoch_acc,
                "epoch": epoch
                })
            
            else:
                wandb.log({
                "val/loss":epoch_loss,
                "val/accuracy":epoch_acc,
                "epoch": epoch})

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if epoch_loss < best_val_loss:
            counter = 0
            best_val_loss = epoch_loss

        else:
            counter += 1

        if counter > patience:
            break

        scheduler.step()


        save_path = os.path.join(checkpoint_path, f"{model_name}_{dataset_name}_epoch{epoch}.pth")
        os.makedirs(save_dir, exist_ok=True)
        # Save the model weights
        torch.save(model.state_dict(), save_path)

    print('Training complete')
    return model

# Execute the training function
model_ft = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=11)


Epoch 1/11
----------
Entered train mode
train Loss: 3.2584 Acc: 0.1956
Entered val mode
val Loss: 2.6523 Acc: 0.2721
Epoch 2/11
----------
Entered train mode


KeyboardInterrupt: 